<a href="https://colab.research.google.com/github/yunmokoo/Hands-On_Machine_Learning/blob/main/Copy_of_Ch2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Housing Example.

In [5]:
from pathlib import Path
import pandas as pd
import tarfile
import urllib.request

# Downloading the Data.

Normally, the dtata would be in relational database or other common data store, and across multiple tables/ documents/ files.

*In this project however,*
just downloading a single compressed file, ***housing.tgz***, will do it.

***housing.tgz*** contains a comma-separated values(CSV) file called, ***housing.csv***, with all the data.

In [6]:
def load_housing_data():
    tarball_path = Path("datasets/housing.tgz")
    if not tarball_path.is_file():
        Path("datasets").mkdir(parents=True, exist_ok=True)
        url = "https://github.com/ageron/data/raw/main/housing.tgz"
        urllib.request.urlretrieve(url, tarball_path)
        with tarfile.open(tarball_path) as housing_tarball:
            housing_tarball.extractall(path="datasets")
    return pd.read_csv(Path("datasets/housing/housing.csv"))

housing = load_housing_data()